# 모듈 부르기

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import warnings
warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

import missingno as msno

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data_dir = os.getenv('HOME')+'/aiffel/Exploration/1st_hackathon' 

train_data_path = join(data_dir, 'sales_train.csv') 
# sub_data_path = join(data_dir, 'test.csv')

In [4]:
data = pd.read_csv(train_data_path)
print('train data dim : {}'.format(data.shape))

train data dim : (2935849, 6)


 data 개수는 300만개가 살짝 안되고, 6가지 컬럼이 있다.

In [5]:
print(data.columns)

Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price',
       'item_cnt_day'],
      dtype='object')


In [6]:
data.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


# 모양바꾸기

In [8]:
data_ndarray = data.values
data_ndarray

array([['02.01.2013', 0, 59, 22154, 999.0, 1.0],
       ['03.01.2013', 0, 25, 2552, 899.0, 1.0],
       ['05.01.2013', 0, 25, 2552, 899.0, -1.0],
       ...,
       ['14.10.2015', 33, 25, 7459, 349.0, 1.0],
       ['22.10.2015', 33, 25, 7440, 299.0, 1.0],
       ['03.10.2015', 33, 25, 7460, 299.0, 1.0]], dtype=object)

In [9]:
del data['date']
del data['date_block_num']
del data['shop_id']
del data['item_cnt_day']

In [10]:
data.head()

,item_id,item_price
0,22154,999.00
1,2552,899.00
2,2552,899.00
3,2554,1709.05
4,2555,1099.00


In [11]:
data_ndarray = data.values
data_ndarray

array([[22154.,   999.],
       [ 2552.,   899.],
       [ 2552.,   899.],
       ...,
       [ 7459.,   349.],
       [ 7440.,   299.],
       [ 7460.,   299.]])

In [12]:
data_dict=data.to_dict()
data_dict

{'item_id': {0: 22154,
  1: 2552,
  2: 2552,
  3: 2554,
  4: 2555,
  5: 2564,
  6: 2565,
  7: 2572,
  8: 2572,
  9: 2573,
  10: 2574,
  11: 2574,
  12: 2574,
  13: 2574,
  14: 2574,
  15: 2574,
  16: 2574,
  17: 2574,
  18: 2574,
  19: 2574,
  20: 2593,
  21: 2604,
  22: 2604,
  23: 2607,
  24: 2607,
  25: 2609,
  26: 2548,
  27: 2611,
  28: 2546,
  29: 2515,
  30: 2705,
  31: 2705,
  32: 2705,
  33: 2715,
  34: 2715,
  35: 2716,
  36: 2716,
  37: 2719,
  38: 2719,
  39: 2719,
  40: 2720,
  41: 2473,
  42: 2480,
  43: 2731,
  44: 2731,
  45: 2746,
  46: 2746,
  47: 2746,
  48: 2746,
  49: 2746,
  50: 2746,
  51: 2746,
  52: 2748,
  53: 2748,
  54: 2748,
  55: 2522,
  56: 2614,
  57: 2614,
  58: 2798,
  59: 2833,
  60: 2833,
  61: 2833,
  62: 2833,
  63: 2833,
  64: 2833,
  65: 2835,
  66: 2835,
  67: 2835,
  68: 2836,
  69: 2836,
  70: 2844,
  71: 2844,
  72: 2844,
  73: 2845,
  74: 2847,
  75: 2847,
  76: 2848,
  77: 2854,
  78: 2854,
  79: 2854,
  80: 2857,
  81: 2857,
  82: 2920,
  

dict보다 ndarray 를 처리하는 방법을 찾아보자

# 간단한 전처리 
각 변수들에 대해 결측 유무를 확인하고, 분포를 확인해보면서 간단하게 전처리를 하겠습니다.
# 결측치 확인
먼저 데이터에 결측치가 있는지를 확인하겠습니다.
missingno 라이브러리의 matrix 함수를 사용하면, 데이터의 결측 상태를 시각화를 통해 살펴볼 수 있습니다.

In [ ]:
msno.matrix(data)

모든 변수에 결측치가 없는 것으로 보이지만, 혹시 모르니 확실하게 살펴보겠습니다.

In [ ]:
# 직접 결측치의 개수를 출력해서 확인
for c in data.columns:
    print('{} : {}'.format(c, len(data.loc[pd.isnull(data[c]), c].values)))

# 각 변수들의 분포 확인
한쪽으로 치우친 분포는 모델이 결과를 예측하기에 좋지 않은 영향을 미치므로 다듬어줄 필요가 있습니다.

In [ ]:
# 가로스크롤 때문에 그래프 확인이 불편하다면 figsize의 x값을 조절해 보세요. 
fig, ax = plt.subplots(3, 2, figsize=(12, 12))  

# id 변수(count==0인 경우)는 제외하고 분포를 확인합니다.
count = 1
columns = data.columns
for row in range(6):
    for col in range(2):
        print(row, col, count)
        sns.kdeplot(data[columns[count]], ax=ax[row][col])
        ax[row][col].set_title(columns[count], fontsize=15)
        count += 1
        if count == 5 :
            break

# log로 치우침 줄이기 할라했는데 왜 안될까

In [ ]:
skew_columns = ['item_price', 'item_cnt_day']

for c in skew_columns:
    data[c] = np.log1p(data[c].values)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10, 15))

count = 0
for row in range(2):
    for col in range(2):
        if count == 5:
            break
        sns.kdeplot(data[skew_columns[count]], ax=ax[row][col])
        ax[row][col].set_title(skew_columns[count], fontsize=15)
        count+=1

# 변수 지정
위에서 저장해두었던 train_len을 인덱스로 활용해서 :train_len까지는 학습 데이터, 즉 x에 저장하고,  
train_len: 부터는 실제로 추론을 해야 하는 테스트 데이터, 즉 sub 변수에 저장합니다  

In [ ]:
sub = data.iloc[train_len:, :]
x = data.iloc[:train_len, :] 

print(x.shape)
print(sub.shape)